## 前言

这个trick来自https://www.coursera.org/learn/competitive-data-science/home/welcome

## Download Data

download data from [this link](https://habrastorage.org/storage/stuff/special/beeline/00.beeline_bigdata.zip)

download data shell script

In [13]:
# !mkdir -p ./data
# !mkdir -p ./data/exploring_anonymized_data-year
# !curl -L https://habrastorage.org/storage/stuff/special/beeline/00.beeline_bigdata.zip -o ./data/exploring_anonymized_data-year.zip
# !unzip -u ./data/exploring_anonymized_data-year.zip -d ./data/exploring_anonymized_data-year/

clean data shell script

In [21]:
# !rm ./data/exploring_anonymized_data-year.zip
# !rm -r ./data/exploring_anonymized_data-year/

In [22]:
# coding: utf8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [23]:
df = pd.read_csv('./data/exploring_anonymized_data-year/train.csv')

In [25]:
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x53,x54,x55,x56,x57,x58,x59,x60,x61,y
0,b4d8a653ea,16a14a2d17,06330986ed,ca63304de0,a62168d626,1746600cb0,1,1,-0.688706,7e5c97705a,...,0.235333,0.264952,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,9.0,2
1,467f9617a3,16a14a2d17,06330986ed,ca63304de0,b7584c2d52,1746600cb0,1,1,0.870871,5624b8f759,...,1.782346,1.322409,0.011647,0.397671,0.239601,0.249584,0.068220,0.033278,601.0,4
2,190436e528,16a14a2d17,06330986ed,ca63304de0,b7584c2d52,1746600cb0,1,1,0.437655,5624b8f759,...,-0.586540,0.672436,0.000000,0.606061,0.121212,0.212121,0.060606,0.000000,33.0,3
3,43859085bc,16a14a2d17,06330986ed,ca63304de0,a62168d626,1746600cb0,1,1,0.004439,f67f142e40,...,-1.600326,-1.838680,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.0,4
4,a4c3095b75,16a14a2d17,06330986ed,ca63304de0,b7584c2d52,1746600cb0,1,1,0.480977,7e5c97705a,...,0.960991,0.790990,0.020161,0.645161,0.258065,0.036290,0.040323,0.000000,248.0,3


## Random Forest

做一个简单的随机森林模型看看

In [32]:
X = df.iloc[:, :-1].copy()
y = df['y']

# fill na
X = X.fillna(-999)

# label encoding
cat_feats = X.columns[X.dtypes == 'object']
X[cat_feats] = X[cat_feats].apply(lambda x: x.factorize()[0])

In [33]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [39]:
pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False).head()

x8     0.237504
x57    0.044805
x56    0.038837
x58    0.033722
x30    0.027581
dtype: float64

可以看到x8的重要程度最高，所以下面我们研究一下x8

In [40]:
df['x8'].head()

0   -0.688706
1    0.870871
2    0.437655
3    0.004439
4    0.480977
Name: x8, dtype: float64

In [42]:
print('x8 mean:', df['x8'].mean())
print('x8 std:', df['x8'].std())

x8 mean: -0.0002523520286219452
x8 std: 1.0232816360125667


可以看到x8的平均值接近0，标准差接近1，估计这个指标应该是做过标准化了

In [43]:
df['x8'].value_counts()

-2.984750     2770
 0.480977     2569
 0.610941     1828
 0.654263     1759
 0.567620     1746
 0.697585     1691
 0.524298     1639
 0.740906     1628
 0.394333     1610
 0.437655     1513
 0.351012     1450
 0.264369     1429
 0.307690     1401
 0.221047     1372
 0.784228     1293
 0.047761     1290
 0.177725     1276
 0.091082     1250
 0.134404     1213
-0.038882     1085
 0.004439     1080
-0.082204     1006
-0.168847      995
-0.125526      976
-0.212169      954
-0.342134      923
-0.385455      921
-0.255490      906
 0.827549      893
-0.298812      881
              ... 
-1.771746        3
 1.130800        3
-2.248283        3
 1.087479        2
-1.945032        2
-2.118319        2
-2.551534        2
-1.988354        2
 1.390730        2
 1.607338        2
 1.304087        2
 1.434051        2
 1.477373        2
 1.347408        2
-1.815068        2
 1.823946        1
-2.161640        1
-2.031675        1
-2.724821        1
 1.564016        1
 1.520695        1
-2.334927   

In [50]:
x8_unique = df['x8'].unique()
x8_unique.sort()
np.diff(x8_unique)

array([43.27826527, 38.98942817,  0.21660793,  0.04332159,  0.17328635,
        0.21660793,  0.08664317,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04332159,
        0.04332159,  0.04332159,  0.04332159,  0.04332159,  0.04

提取x8所有的值，并做diff，查看两两之间的差

可以看到中间部分两两之间的差基本都是0.04332159

那么我们可以这个假设，这些值原本的间隔是1，进行标准化后变成0.04332159

那么我们将这个间隔变回去

In [51]:
np.diff(x8_unique / 0.04332159)

array([998.99992752, 899.9999347 ,   4.99999964,   0.99999993,
         3.99999971,   4.99999964,   1.99999985,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99999993,
         0.99999993,   0.99999993,   0.99999993,   0.99

In [52]:
(df['x8'] / 0.04332159).value_counts()

-68.897526      2770
 11.102468      2569
 14.102468      1828
 15.102468      1759
 13.102468      1746
 16.102468      1691
 12.102468      1639
 17.102468      1628
 9.102468       1610
 10.102468      1513
 8.102469       1450
 6.102469       1429
 7.102469       1401
 5.102469       1372
 18.102468      1293
 1.102469       1290
 4.102469       1276
 2.102469       1250
 3.102469       1213
-0.897531       1085
 0.102469       1080
-1.897531       1006
-3.897531        995
-2.897531        976
-4.897531        954
-7.897530        923
-8.897530        921
-5.897530        906
 19.102468       893
-6.897530        881
                ... 
-40.897528         3
-39.897528         3
 26.102467         3
 32.102467         2
-48.897527         2
-44.897528         2
 31.102467         2
-45.897528         2
 33.102467         2
 30.102467         2
 37.102466         2
-41.897528         2
 34.102467         2
-58.897527         2
 25.102467         2
 27.102467         1
-62.897526   

观察上述数据，我们再将x8减去0.102469

In [53]:
(df['x8'] / 0.04332159 - 0.102469).value_counts()

-6.899999e+01    2770
 1.100000e+01    2569
 1.400000e+01    1828
 1.500000e+01    1759
 1.300000e+01    1746
 1.600000e+01    1691
 1.200000e+01    1639
 1.700000e+01    1628
 8.999999e+00    1610
 9.999999e+00    1513
 8.000000e+00    1450
 6.000000e+00    1429
 7.000000e+00    1401
 5.000000e+00    1372
 1.800000e+01    1293
 1.000000e+00    1290
 4.000000e+00    1276
 2.000000e+00    1250
 3.000000e+00    1213
-9.999998e-01    1085
 1.277230e-07    1080
-2.000000e+00    1006
-4.000000e+00     995
-3.000000e+00     976
-5.000000e+00     954
-7.999999e+00     923
-8.999999e+00     921
-5.999999e+00     906
 1.900000e+01     893
-6.999999e+00     881
                 ... 
-4.100000e+01       3
-4.000000e+01       3
-5.200000e+01       3
 3.200000e+01       2
 2.500000e+01       2
-4.900000e+01       2
-4.600000e+01       2
 3.400000e+01       2
-4.200000e+01       2
 3.300000e+01       2
 3.100000e+01       2
 3.000000e+01       2
 3.700000e+01       2
-4.500000e+01       2
-5.900000e

In [54]:
new_x8 = df['x8'] / 0.04332159 - 0.102469

In [56]:
new_x8.describe()

count    48793.000000
mean        -0.108294
std         23.620593
min      -1967.999857
25%         -5.999999
50%          5.000000
75%         12.999999
max         78.999994
Name: x8, dtype: float64

看到-1968这个值，感觉很像年进行平移后的值，再加个1968试试

In [59]:
(new_x8 + 1968).value_counts().sort_index()

0.000143          1
999.000070        4
1899.000005    2770
1904.000005       1
1905.000005       1
1909.000004       2
1914.000004       1
1916.000004       3
1917.000004       1
1918.000004       1
1919.000004       2
1920.000004       1
1921.000004       1
1922.000003       2
1923.000003       2
1924.000003       4
1925.000003       4
1926.000003       2
1927.000003       3
1928.000003       3
1929.000003       4
1930.000003       4
1931.000003      12
1932.000003      10
1933.000003       7
1934.000003      13
1935.000003      28
1936.000002      35
1937.000002      35
1938.000002      45
               ... 
1977.999999    1513
1978.999999    2569
1979.999999    1639
1980.999999    1746
1981.999999    1828
1982.999999    1759
1983.999999    1691
1984.999999    1628
1985.999999    1293
1986.999999     893
1987.999999     624
1988.999999     434
1989.999999     233
1990.999998     110
1991.999998      31
1992.999998       2
1993.999998       3
1994.999998       1
1997.999998       2
